---

In [ ]:
            if self.classifier:
                classifier_logits = self.classifier(self._get_obs(update_history=False)['history']['history'])
                new_dist = torch.distributions.Categorical(logits=classifier_logits)
                kl_bad = torch.distributions.kl.kl_divergence(self.whole_dist, new_dist)
                kl_good = torch.distributions.kl.kl_divergence(self.prev_dist, new_dist)
                reward = kl_good.item() - .3
                self.prev_dist = new_dist
                self.history.process_all()
                if kl_bad < .5:
                    end = True
                else:
                    end = False
            else:
                reward = self._reward_missed_soft()
                reward = np.clip(reward, -10, 10)
                end = True
                
            logs = self.get_logs(reward, True)
            return self._get_obs(update_history=False), reward, end, False, {'logs':logs}

In [ ]:
from transformers import AutoImageProcessor, ViTMAEModel, ViTMAEConfig
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-base')
config = ViTMAEConfig.from_pretrained('facebook/vit-mae-base',
                                             proxies={'http': '127.0.0.1:10809', 'https': '127.0.0.1:10809'},)
model = ViTMAEModel.from_pretrained('facebook/vit-mae-base', config=config)

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
# loss = outputs.loss
# mask = outputs.mask
# ids_restore = outputs.ids_restore

In [ ]:
from multiprocessing import Array, Pipe, connection
parent_conn, child_conn = Pipe()

In [ ]:
from multiprocessing.context import Process
from multiprocessing import Array, Pipe, connection, SimpleQueue

def classifier_tgt(pipe):
    classifer = ViTClassifier().to(device2)
    while True:
        try:
            x = pipe.recv()
            out = classifer(x)
            pipe.send(out)
        except EOFError:
            print('closed'*20)
            pipe.close()
            break

if __name__ == "__main__":
    parent_conn, child_conn = Pipe()
    p = Process(target=classifier_tgt, args=(child_conn,), daemon=True)
    p.start()
    # child_conn.close()

def surrogate_classifier(x):
    parent_conn.send(x)
    return parent_conn.recv()

if __name__ == "__main__":
    surrogate_classifier.device = device2
    print(surrogate_classifier(torch.ones(3, 224, 224)).shape) #! neeeeeeeeeed RPC

---

In [ ]:
dataset = COCODataset(train=True, indices=[
                        4385], no_seg=False, fix_resize=(224-3*16, 224-3*16))
env = TimeLimit(Environment(dataset,
                            config['env_patch_size'],
                            max_len=config['env_step_mem'],
                            n_last_positions=config['n_last_positions']
                            ),
                config['env_step_limit_test'])
obs, _ = env.reset()
display.clear_output()


In [ ]:
obs, *_ = env.step(Actions.LEFT)

In [ ]:
batch = ts.data.Batch(
                obs=[obs],
                info=None
)

In [ ]:
policy.actor.last.model

Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
  (4): Linear(in_features=256, out_features=4, bias=True)
)

In [ ]:
def fn(model, before, after):
    print(before[0], len(before))
hook.remove()
hook = policy.actor.last.model[4].register_forward_hook(fn)

In [ ]:
batch_out = policy(batch)

In [ ]:
batch_out.logits

tensor([[ -2.9156,  11.7836,  -2.5068, -14.9789]], device='cuda:1',
       grad_fn=<AddmmBackward0>)

In [ ]:
logitsg = classifier(env.current_image.unsqueeze(0).to(device2))
probsg = torch.softmax(logitsg, -1)
print(classifier.vit.config.id2label[logitsg.argmax().item()], probsg.max().item())

meat loaf, meatloaf 0.5452646613121033


In [ ]:
logits = classifier(out[0]['history']['history'].to(torch.float32).unsqueeze(0).to(device2))
print(classifier.vit.config.id2label[logits.argmax().item()], torch.softmax(logits, -1).max().item(), probsg[0, logits.argmax().item()].item())